In [7]:
include("functions.jl")

magic = readdlm("magic04.data", ',', Any, '\n')
magic = magic[sample(1:19020, 200, replace = false),:]
loc = findall(x->x=="g",magic[:,11])
magic[loc,11] .= 1
loc = findall(x->x=="h",magic[:,11])
magic[loc,11] .= 0
magic = convert(Array{Float64}, magic)

class = magic[:,11]
magic[:,11] .= 1
n, p = size(magic)

(200, 11)

Magic gamma telescope dataset
- \# of sample: 19,020
- \# of covariate: 11 (including intercept)
- response: binary

Choose $N_1$ random samples, and obtain $\hat\beta$ for the logistic regression model. Then, find a locally {$A_K$, $D$}-optimal design of sample size $N_2$ with $\hat\beta$.

#### Case 1, TED
- $A_K$-optimality
- $N_1$: 30
- $N_2$: 100

In [13]:
using BenchmarkTools, StatsBase

Random.seed!(1)
N1 = 30
N2 = 100
samp1 = sample(1:n, N1, replace = false)
cand = setdiff(1:n, samp1)

@time aopt = sagnol_A(magic[cand,:], magic[samp1,:], N2; K = magic', verbose=1, IC=1)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 42887           
  Cones                  : 171             
  Scalar variables       : 80856           
  Matrix variables       : 0               
  Integer variables      : 171             

Optimizer started.
Mixed integer optimizer started.
Threads used: 24
Presolve started.
Presolve terminated. Time = 1.39
Presolved problem: 40682 variables, 2542 constraints, 816279 non-zeros
Presolved problem: 0 general integer, 170 binary, 40512 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        0        0        NA                   -3.0308183564e-01    NA          4.7   
0        1        0        0        -2.9818481763e-01    -3.0308183564e-01    1.64        22.9  
0        1        0        0        -3.0287075677e-

170-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0

In [14]:
@time aopt = sagnol_A(magic[cand,:], magic[samp1,:], N2; K = magic', verbose=1, IC=0)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 43058           
  Cones                  : 171             
  Scalar variables       : 80856           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 171
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.03            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.31    
Problem
  Name                   :                 
  Objective 

┌ Warning: Problem status Stall; solution may be inaccurate.
└ @ Convex /DATA/home/ppinsm/.julia/packages/Convex/6NNC8/src/solution.jl:51


170-element Array{Float64,1}:
 0.8879621453633537    
 0.9999648928258077    
 0.9998609379995613    
 0.9999589387231197    
 6.941027938368997e-5  
 0.9999275656076103    
 0.0001721885915834619 
 8.888938581606475e-5  
 0.11192619146478118   
 7.678611206766448e-5  
 0.00031226427559011445
 0.00016611892738583914
 7.491940688676624e-5  
 ⋮                     
 0.9999887666152294    
 0.9999554783830741    
 0.00013759134689405716
 7.29448885497251e-5   
 0.9999913816214989    
 0.9999645973860484    
 7.130257257343016e-5  
 0.13663600083345256   
 0.9999380705736722    
 0.9999756282229514    
 0.9972848834887176    
 8.605691771128575e-5  

In [15]:
#N1 = 30
#N2 = 100
#samp1 = sample(1:n, N1, replace = false)

@time aopt = sagnol_A(magic[cand,:], magic[samp1,:], N2; K = magic', verbose=0, IC=1)
aopt = BitArray(round.(aopt))
println(sum(aopt))
samp2 = [samp1; cand[aopt]]
exact = tr(magic*((magic[samp2,:]'magic[samp2,:])\(magic')))

Optimal
 56.415248 seconds (2.28 M allocations: 18.662 GiB, 3.70% gc time)
100


13.198777128972031

In [16]:
@time aopt = sagnol_A(magic[setdiff(1:n, samp1),:], magic[samp1,:], N2; K = magic', verbose=0, IC=0)

n_try = 30

apprx = zeros(Float64, n_try)
apprx_size = zeros(Int64, n_try)
for i = 1:n_try
    bool_opt = rand(n-N1) .< aopt
    apprx_size[i] = sum(bool_opt)
    samp2 = [samp1; cand[bool_opt]]
    apprx[i] = tr(magic*((magic[samp2,:]'magic[samp2,:])\(magic')))
end

countmap(apprx_size)

Stall
 30.222466 seconds (2.25 M allocations: 18.669 GiB, 7.21% gc time)


┌ Warning: Problem status Stall; solution may be inaccurate.
└ @ Convex /DATA/home/ppinsm/.julia/packages/Convex/6NNC8/src/solution.jl:51


Dict{Int64,Int64} with 5 entries:
  100 => 9
  102 => 2
  98  => 2
  101 => 8
  99  => 9

In [17]:
describe(exact ./ apprx)

Summary Stats:
Length:         30
Missing Count:  0
Mean:           0.999666
Minimum:        0.991286
1st Quartile:   0.995677
Median:         0.999820
3rd Quartile:   1.004053
Maximum:        1.008297
Type:           Float64
